# Práctica 1: Criptografía clásica
## UAM, 2022/2023

### Maitane Gómez González
### Ana Martínez Sabiote

## 1. Sustitución monoalfabeto

## 1.a Método afín
El siguiente programa implementa el método afín.



In [1]:
import gmpy2
from gmpy2 import mpz
import sympy
import numpy as np

"""
VERSIÓN SIN GMP 

def algoritmo_euclides(a,b):
    if a%b == 0:
        return b
    else:
        return algoritmo_euclides(b, a%b)
"""

Función que cálcula el máximo común divisor entre dos números pasados por parámetro (a y b)

In [2]:
def algoritmo_euclides(a,b):
    if gmpy2.t_mod(a,b) == 0:
        return b
    else:
        return algoritmo_euclides(b, gmpy2.t_mod(a,b))

In [3]:
mcd=algoritmo_euclides(39,150)
print(mcd)

3


In [4]:
algoritmo_euclides(7,15)

mpz(1)

"""
VERSIÓN SIN GMP 

def algoritmo_euclides_extendido(a,b):

    # Identidad de Bézout 1=u*a + v*b
    # El inverso de a módulo b es u. Recíprocamente, el inverso de b mod a es v
    if a==0:
        mcd=b
        u=0
        v=1
    else:
        mcd, x, y = algoritmo_euclides_extendido(b%a, a)
        u=y-(b//a)*x
        v=x
        
    return mcd, u, v
"""

In [5]:
def algoritmo_euclides_extendido(a,b):
    """
    # Condición a>b, sino las cambiamos
    if b>a:
        aux=a
        a=b
        b=aux
    """
    # Identidad de Bézout 1=u*a + v*b
    # El inverso de a módulo b es u. Recíprocamente, el inverso de b mod a es v
    if a==0:
        mcd=b
        u=0
        v=1
    else:
        mcd, x, y = algoritmo_euclides_extendido(gmpy2.c_mod(b,a), a)
        u=gmpy2.sub(y,(gmpy2.mul(gmpy2.c_div(b,a),x)))
        v=x
        
    return mcd, u, v

In [6]:
def inverso(a,m):
    result = algoritmo_euclides_extendido(a,m)
    # Comprobamos que el mcd es 1 para que exista inverso multiplicativo
    # En consecuencia, a y m determinan una función afín inyectiva
    if result[0] == 1:
        # Entonces devolvemos el coeficiente u (que acompaña a) de la Id. de Bézout
        inv=result[1]
        return inv
    else:
        print("Error")

In [7]:
inverso(51,23)

mpz(-9)

In [8]:
def read_input(i):
    # Primero tomamos el input de i o de la entrada estándar
    if i==0:
        cadena=input()
    else:
        file=open(i, "r")
        cadena=file.read()
        file.close()
    print("Cadena: {}".format(cadena))
    return cadena

In [9]:
def print_output(o,cadena):
    if o==0:
        print("Cadena: {}".format(cadena))
    else:
        file=open(o, "w")
        cadenaToStr = ' '.join([str(elem) for elem in cadena])
        file.write(cadenaToStr)
        file.close()

Llamada a la función:

afin {-C|-D} {-m |Zm|} {-a N×} {-b N+} [-i filein] [-o fileout]


In [10]:
def afin(modo,m,a,b,i=0,o=0):
    alfabeto='abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ'
    if algoritmo_euclides(a,m) == 1:
        if modo=="-C":
            cadena=read_input(i)
            #Traducimos los caracteres a números
            cadena_numerica=[]
            for k in cadena:
                if k in alfabeto: 
                    cadena_numerica.append(alfabeto.index(k))
            cadena_cifrada=[]
            for k in cadena_numerica:
                cadena_cifrada.append(((a*k)+b)%m)
            
            #print(cadena_cifrada)
            resul=""
            for i in range(len(cadena_cifrada)):
                   resul=resul+alfabeto[cadena_cifrada[i]]
                    
            print_output(o,resul)
            
        elif modo=="-D":
            
            cadena_cifrada=read_input(i)
            #cadena_cifrada=cadena_cifrada.split(", ")
            
            cadena_numerica=[]
            for k in cadena_cifrada:
                if k in alfabeto: 
                    cadena_numerica.append(alfabeto.index(k))
                    
            cadena_descifrada=[]
            cadena_texto=""
            #inv=inverso(m,a)
            inv=pow(a, -1, m)
            print(inv)
            for i in range(len(cadena_numerica)):
                cadena_numerica[i]=int(cadena_numerica[i])
                
            for k in cadena_numerica:
                k_descifrado=gmpy2.c_mod(gmpy2.mul((k-b),inv),m)
                if k_descifrado<0:
                    k_descifrado=m+k_descifrado
                cadena_descifrada.append(k_descifrado)
                cadena_texto=cadena_texto+alfabeto[k_descifrado]
            
            print_output(o, cadena_texto)
    else:
        print("{} y {} no son primos relativos. Error".format(a,m))

In [11]:
afin("-C",51,23,3,"cadena.txt","cadena_cifrada.txt")

Cadena: Hola Nueva York!


In [12]:
afin("-D",51,23,3, "cadena_cifrada.txt")

Cadena: W t b d H e S B d F t L D
20
Cadena: HolaNuevaYork


In [13]:
afin("-C",130,16,27)

16 y 130 no son primos relativos. Error


In [14]:
afin("-C",51,13,0)

hola
Cadena: hola
Cadena: ODPa


In [15]:
afin("-D",51,13,0)

holiss
Cadena: holiss
4
Cadena: CfSGvv


## 1.b Criptoanálisis del cifrado afín

Nuestro afin no trivial se basa en aumentar el tamaño de la clave. Hemos decidido cambiar el tamaño de la base y cifrarlo en bigramas. Lo que nos daría un espacio de 676.Con el afín normal, en nuestro caso, obteniamos uno de 52. 

Como se puede observar en la función #fortaleza y en su ejecución el normal, nos daría 128 claves y el no trivial 11545444563871328761349212098135488565445348609393477048015277366400000000. 



In [16]:
def fortaleza(m):
    z_m_inv=sympy.totient(m) #calculamos la funcion phi
    return gmpy2.mul((m),z_m_inv) #calculamos la fortaleza multiplicando Zm y Zm*


In [661]:
print(fortaleza(26**26+26))
#fortaleza(26) este seria el resultado si no aceptaramos mayúsculas
print(fortaleza(52))


11545444563871328761349212098135488565445348609393477048015277366400000000
1248


El siguiente programa implementa el método afín no trivial.

Llamada a la función:

afin_no_trivial {-C|-D} {-m |Zm|} {-a N×} {-b N+} [-i filein] [-o fileout]

- -C el programa cifra
- -D el programa descifra
- -m tamaño del espacio de texto cifrado
- -a coeficiente multiplicativo de la función afín
- -b término constante de la función afín
- -i fichero de entrada
- -o fichero de salida

En este programa utilizamos un alfabeto de 26 elementos, las letras del abecedario en minúsculas


In [19]:
def afin_no_trivial(modo,m,a,b,i=0,o=0):
    alfabeto='abcdefghijklmnopqrstuvwxyz'
    digrama=([]) #generamos un vetor de di-gramas del alfabeto que hemos declarado arriba
    for k in alfabeto:
        for j in alfabeto:
            digrama.append(k+j)
    
    if algoritmo_euclides(a,m) == 1:
        if modo=="-C":
            #obtenemos el texto claro, si no se ha pasado por parámetro, se obtine de teclado
            cadena=read_input(i)

            
            #ponemos la cadena de entrada solo en caracteres de nuestro alfabeto
            cadena_formateada=""
            for k in range(len(cadena)):
                if cadena[k] in alfabeto:
                    cadena_formateada=cadena_formateada+cadena[k]
                
            #Traducimos los caracteres a números para poder operar
            cadena_numerica=[]
            j=0
            if len(cadena_formateada)%2==0:
                    while j <(len(cadena_formateada)-1):
                        cadena_numerica.append(digrama.index(cadena_formateada[j]+cadena_formateada[j+1]))
                        j=j+2
            else:
                    while j <(len(cadena_formateada)-2):
                        cadena_numerica.append(digrama.index(cadena_formateada[j]+cadena_formateada[j+1]))
                        j=j+2
                    cadena_numerica.append(alfabeto.index(cadena_formateada[len(cadena_formateada)-1])) #como es impar, la última letra la ciframos aparte
            
            #utilizamos la función de cifrado
            cadena_cifrada=[]
            for k in cadena_numerica:
                cadena_cifrada.append(((a*k)+b)%m)
            
            #pasamos el resultado a caracteres y lo guardamos como string
            resul=""
            for k in cadena_cifrada:
                resul=resul+digrama[k]
          
            #si no se ha pasado un fichero de salida por parámetro, imprimimos el resultado
            print_output(o,resul)
            
        elif modo=="-D":
            
            cadena_cifrada=read_input(i)
            #cadena_cifrada=input()
    
            cadena_descifrada=[]
            cadena_texto=""
            
             #obtenemos el texto claro, si no se ha pasado por parámetro, se obtine de teclado
            #if i==0:
             #    cadena_cifrada=input()
            #cadena=read_input(i)
            #else:
             #   file=open(i, "r")
             #   cadena_cifrada=file.read()
              #  file.close()
                
            #obtenemos el inverso en el modulo para poder utilizar la función de descifrado
            #ya hemos comprobado al principio que el inverso existe.
           
            #inv=inverso(a,m)
            inv=pow(a, -1, m)
            
            #pasamos el texto a un formato numérico para poder operar
            cadena_numerica=[]
            j=0
            if len(cadena_cifrada)%2==0:
                while j <(len(cadena_cifrada)-1):
                    cadena_numerica.append(digrama.index(cadena_cifrada[j]+cadena_cifrada[j+1]))
                    j=j+2
            else:
                while j <(len(cadena_cifrada)-2):
                    cadena_numerica.append(digrama.index(cadena_cifrada[j]+cadena_cifrada[j+1]))
                    j=j+2
                cadena_numerica.append(alfabeto.index(cadena_cifrada[len(cadena_cifrada)-1])) #como es impar, la última letra la ciframos aparte
            
            #desciframos el texto con la función de descifrado: 
            for k in cadena_numerica:
                k_descifrado=gmpy2.c_mod(gmpy2.mul((k-b),inv),m)
                if k_descifrado<0: #ajustamos el modulo
                    k_descifrado=m+k_descifrado
                cadena_descifrada.append(k_descifrado)
            
            #pasamos el texto a caracteres
            if len(cadena_cifrada)%2==0:
                for k in cadena_descifrada:
                    cadena_texto=cadena_texto+digrama[k]
            else:       
                for k in range(len(cadena_descifrada)-1):
                    cadena_texto=cadena_texto+digrama[cadena_descifrada[k]]
                    
                cadena_texto=cadena_texto+alfabeto[cadena_descifrada[(len(cadena_descifrada)-1)]]
            
            #si no se ha pasado un archivo para guardar el resultado, se imprime por pantalla
            print_output(o, cadena_texto)
    else:
        print("{} y {} no son primos relativos. Error".format(a,m))

In [20]:
afin_no_trivial("-C",701,23,3)

palabra
Cadena: palabra
Cadena: vpkmlfad


In [810]:
afin_no_trivial("-C",701,23,3, "cadena_trivial.txt")

Cadena: hola super nueva york
Cadena: ltkmalzdzefiyuzhwn


In [811]:
afin_no_trivial("-D",701,23,3, 0, "resultado_trivial.txt")

KeyboardInterrupt: Interrupted by user

In [666]:
afin_no_trivial("-D",701,23,3)

ltkmalzdzefiyuzhwn
Cadena: ltkmalzdzefiyuzhwn
Cadena: holasupernuevayork


El cifrado afin es muy vulnerable a los ataques. Se rompe imediatamente con B,C,D y E. Con A hace falta un analisis de frecuencias (El análisis de frecuencia es el estudio de la frecuencia de letras o grupos
de letras en un texto cifrado).

#### Ejemplo de criptoánalisis afín

Hemos cifrado "antiaereo" con afin (modulo 51, a=13 y b=0), lo que nos ha dado la cadena "aqRcabrbD". 

En el ejemplo de abajo hemos guardado las tablas de frecuencia del castellano y el ingles y luego las hemos ordenado por mayor a menor. En este caso solo hemos utilizado la del castellano.

Hemos conseguido descifrarlo con la segunda hipotesís:
c1: la posición del elemento más utilizado de la cadena en el alfabeto.
c2: la posición del segundo elemento más utilizado de la cadena en el alfabeto.
t1: la posición del elemento más utilizado en el alfabeto.
t2: la posición del segundo elemento más utilizado en el alfabeto.

pos(c1)=pos(t1)* a+b 
pos(c2)=pos(t1)* a+b

Se puede resolver de dos formas:
1- restando las ecuaciones, lo que nos daría el resultado de: 
      pos(c1)-pos(c2)=(pos(t1)-pos(t2))* a -> 
      a=pos(c1)-pos(c2) inv((pos(t1)-pos(t2))
      b=pos(c1)-pos(t1)* a

2- Al introducir los datos conocidos, nos damos cuenta de que se puede simplificar y resolver casi directamente:
      pos(c1)=pos(t1)* a+b -> 1=4* a+b->a=1* inv(4)=13
      pos(c2)=pos(t1)* a+b -> 0=0* a+b ->b=0

En ambos casos el resultado da a=13, b=0. El cifrado esta roto.

In [812]:


cadena = "aqRcabrbD"


castellano=(['a', 11.96],['b', 0.92],['c', 2.92],['d', 6.87],['e', 16.78],['f', 0.52],['g', 0.73],
           ['h', 0.89],['i', 4.15],['j', 0.3],['k', 0.0],['l', 8.37],['m', 2.12],['n', 7.01],
           ['o', 8.69],['p', 2.77],['q', 1.53],['r', 4.94],['s', 7.88],['t', 3.31],['u', 4.80],
           ['v', 0.39],['w', 0.0],['x', 0.06],['y', 1.54],['z', 0.15])

ingles=(['a', 11.96],['b', 1.54],['c', 3.06],['d', 3.99],['e', 12.51],['f', 2.30],['g', 1.96],
        ['h', 0.89],['i', 7.26],['j', 0.16],['k', 0.67],['l', 4.14],['m', 2.53],['n', 7.09],
       ['o', 7.60],['p', 2.0],['q', 0.11],['r', 6.12],['s', 6,54],['t', 9.25],['u', 2.71],
       ['v', 0.99],['w', 1.92],['x', 1.92],['y', 1.73],['z', 0.19])


castellano_ord=sorted(castellano, key=lambda letra: letra[1], reverse=True)
ingles_ord=sorted(ingles, key=lambda letra: letra[1], reverse=True)

print("Primera hipotesis:")
c1=alfabeto.index("a")
c2=alfabeto.index("b")
t1=alfabeto.index(castellano_ord[0][0])
t2=alfabeto.index(castellano_ord[1][0])

#pos(c1)=pos(t1)*a+b -> 0=4*a+b->a=-1*inv(4)
#-
#pos(c2)=pos(t1)*a+b -> 1=0*a+b ->b=1
a=1*pow(-1,-1,51)

#comprobamos que hemos resuelto bien la ecuación
#comprobamos que sean co-primos
if algoritmo_euclides(a,51)==1:
        b=c1-int(a)*t1
        if a==13 and b%51==0:
            print("es correcto, antiaereo se cifro con 13 y 0")
        else:
            print("No es correcto")
else:
    print("no son coprimos")


print("Segunda hipotesis:")
c1=alfabeto.index("b")
c2=alfabeto.index("a")
t1=alfabeto.index(castellano_ord[0][0])
t2=alfabeto.index(castellano_ord[1][0])

#pos(c1)=pos(t1)*a+b -> 1=4*a+b->a=1*inv(4)
#-
#pos(c2)=pos(t1)*a+b -> 0=0*a+b ->b=0

#pos(c1)-pos(c2)=(pos(t1)-pos(t2))*a
#0-1=(4-0)*a
#a=1*inv(4)

a=1*pow(4,-1,51)

#comprobamos que hemos resuelto bien la ecuación
#comprobamos que sean co-primos
if algoritmo_euclides(a,51)==1:
        b=c1-int(a)*t1
        if a==13 and b%51==0:
            print("Es correcto, antiaereo se cifro con 13 y 0")
        else:
            print("No es correcto")
        
else:
    print("no son coprimos")
              
    

Primera hipotesis:
No es correcto
Segunda hipotesis:
Es correcto, antiaereo se cifro con 13 y 0


## 2. Sustitución polialfabeto

## 2.a Método de Hill

El siguiente programa implementa el método hill. El cifrado Hill es de
sustitución poligráfica basado en álgebra lineal.

En este cifrado se utiliza una matriz cuadrada como clave de dimesiones n*n, tenemos que divir el texto claro (ahora numerico) en bloques de n elementos. Si la división no es exacta, se hace padding.

El requisito principal para poder cifrar y descifrar, es que la matriz tenga una función biyectiva. Si no fuera así habría que cambiar los datos ya que si lo cifráramos no podríamos descifrarlo.

Primero se asocia cada letra del alfabeto con un número. La forma más sencilla es hacerlo con la asociación natural ordenada pero se podría hacer mediante otras asociaciones.

Después, aplicamos la función de cifrado y volvemos a pasar el resultado a letras. Para descifrar el proceso es muy parecido, simplemente hay que cambiar la función de cifrado por la de descifrado. Y para ello tenemos que tener la inversa de la matriz calculada.






In [24]:
import numpy as np
import os
import math
import copy

Funcion que cálcula el determinante de una matriz.

determinante{matriz}

In [25]:
def determinante(matriz):
   
    if len(matriz)==2 and len(matriz[0])==2:
        #calculamos el determinante
        det=matriz[0][0]*matriz[1][1]-(matriz[1][0]*matriz[0][1])
       
        return det
    else:
        suma=0
        for i in range(len(matriz)): #calculamos el determinante por cofactores
            maux=copy.deepcopy(matriz)
            maux.remove(matriz[0]) #eliminamos la primera fila
            for j in range(len(maux)):
                maux[j]=maux[j][0:i]+maux[j][i+1:]
                
         
            suma= suma+ (-1)**((i+j)%2)*matriz[0][i]*determinante(maux)
            
        return suma
        

In [26]:
#comprobación de la función
matriz = [[11,8], [3,7]]
print(determinante(matriz))

-8
53


Función que cálcula el adjunto de una matriz.

adjunto{matriz}

In [27]:
def adjunto(matriz):
    adjunto=np.zeros(np.shape(matriz))
    if len(matriz)==2 and len(matriz[0])==2:
         #calculamos el adjunto
        adjunto[0][0]=matriz[1][1]
        adjunto[0][1]=-matriz[0][1]
        adjunto[1][0]=-matriz[1][0]
        adjunto[1][1]=matriz[0][0]
        
        return adjunto
    else:
        
        for i in range(len(matriz)):
            maux=copy.deepcopy(matriz)
            for j in range(len(matriz)):
             
                maux=np.delete(matriz,i,0)
                aux=np.delete(maux,j,1)
                auxi=aux.tolist()
                #la matriz de cofactores transpuesta es el djunto
                adjunto[j][i]=(-1)**((i+j)%2)*determinante(auxi)
            
                
        return adjunto

In [28]:
#comprobación de la función
matriz = [[11,8], [3,7]]
print(adjunto(matriz))

[[ 7. -8.]
 [-3. 11.]]


Función que cálcula la inversa de una matriz.

inversa{matriz}{modulo}

In [29]:
def inversa(matriz,modulo):
    inversa=np.zeros(np.shape(matriz))
    det=determinante(matriz)%modulo
    if det !=0:
        adj=adjunto(matriz)%modulo
        for i in range(len(matriz)):
            for j in range(len(matriz[i])):
                inversa[i][j]=(adj[i][j]/det)#%modulo
                
    return inversa%modulo #esto puede que no sea necesario porque ya estamos en matemática modular

In [30]:
#comprobación de la función
matriz = [[11,8], [3,7]]
print(inversa(matriz,26))

[[ 7. 18.]
 [23. 11.]]


Función de cifrado del algoritmo.

cifrar{matriz_numerica}{matriz}{mod}{n}

Parámetros:
- matriz_numerica: el texto a cifrar en formato matriz de numeros
- matriz: matriz de transformación
- mod: modulo en el que trabajamos
- n: dimensión 

In [635]:
def cifrar(matriz_numerica, matriz,mod,n):
  
    matriz_cifrada=[]
    for i in range(len(matriz_numerica)):
        cadena_cifrada= (np.dot(matriz_numerica[i],matriz))%mod #utilizamos la función de cifrado
        matriz_cifrada.append(cadena_cifrada)

    return matriz_cifrada  

Función de descifrado del algoritmo.

descifrar{matriz_cifrada}{matriz}{mod}{n}

Parámetros:

- matriz_cifrada: el cifrado a descifrar en formato matriz de numeros
- matriz: matriz de transformación
- mod: modulo en el que trabajamos
- n: dimensión

In [649]:
def descifrar(matriz_cifrada, matriz,mod,n):
    
    inv=inversa(matriz,mod)
    matriz_descifrada=[]
    for i in range(len(matriz_cifrada)):
        cadena_descifrada= (np.dot(matriz_cifrada[i],inv))%mod #utilizamos la funcion de descifrado
        matriz_descifrada.append(cadena_descifrada)

    return matriz_descifrada  

Llamada a la función:

hill {-C|-D} {-m |Zm|} {-n NK} {-k f ileK} [-i f ilein] [-o f ileout]

Los parámetros introducidos en este caso son:
-m cardinalidad de Zm
-n dimensión de la matriz de transformación
-k fichero que contiene la matriz de transformación

In [650]:
def hill(modo,mod,n,k,i=0,o=0):
    alfabeto='abcdefghijklmnopqrstuvwxyz'
  
    #leemos la matriz de transformación del archivo y la guardamos
    with open(k,'r') as f:
        datos = ''.join(f.readlines()).replace('\n',';')
    matriz = np.matrix(datos).tolist()
    f.close()
    
    #cálculamos el determinante de la matriz
    det=np.linalg.det(matriz)
    #comprobamos que la matriz K tiene una función biyectiva
    if algoritmo_euclides(int(det),mod)==1:
       
        if modo=="-C":
            
            cadena=read_input(i)
            #Traducimos los caracteres a números
            cadena_numerica=[]
            for k in cadena:
                if k in alfabeto: 
                    cadena_numerica.append(alfabeto.index(k))
                    
            # Dividimos en bloques de n elementos el texto
            # Si m no es múltiplo de n se añade padding
            m=len(cadena_numerica)/n
            maxi=len(cadena_numerica)
            
            matriz_numerica=np.zeros((math.ceil(m),n))
         
            pos=0
            for i in range(math.ceil(m)):
                for j in range(n):
                    if pos<maxi:
                        matriz_numerica[i][j]=cadena_numerica[pos]
                        pos=pos+1
        
            #ciframos cadena a cadena y lo guardamos en un matriz           
            matriz_cifrada=cifrar(matriz_numerica,matriz,mod,n)
            
            #lo volvemos a pasar a caracteres
            resul=""
            for i in range(len(matriz_cifrada)):
                for j in range(len(matriz_cifrada[i])):
                    resul=resul+alfabeto[int(matriz_cifrada[i][j])]
          
            print_output(o,resul)
           
            
        elif modo=="-D":
            if i==0:
                cadena_cifrada=input()
                datos=[]
                for i in range(len(cadena_cifrada)):
                    if cadena_cifrada[i] in alfabeto:
                        datos.append(alfabeto.index(cadena_cifrada[i]))
                
                        
            else:
                file=open(i, "r")
                cadena_cifrada=file.read()
                file.close()
               
                
                datos=[]
                for i in range(len(cadena_cifrada)):
                    if cadena_cifrada[i] in alfabeto:
                        datos.append(alfabeto.index(cadena_cifrada[i]))

             
                
            # Dividimos en bloques de n elementos el texto
            # Si m no es múltiplo de n se añade padding
            m=len(datos)/n
            maxi=len(datos)
            
            matriz_cifrada=np.zeros((math.ceil(m),n))
         
            pos=0
            for i in range(math.ceil(m)):
                for j in range(n):
                    if pos<maxi:
                        matriz_cifrada[i][j]=datos[pos]
                        pos=pos+1
        
            #ciframos cadena a cadena y lo guardamos en un matriz  
            matriz_descifrada=descifrar(matriz_cifrada, matriz,mod,n)
            
            resul=""
            for i in range(len(matriz_descifrada)):
                for j in range(len(matriz_descifrada[i])):
                    
                    if matriz_descifrada[i][j]<0:
                        matriz_descifrada[i][j]=mod+matriz_descifrada[i][j]
        
                    resul=resul+alfabeto[int(matriz_descifrada[i][j])]
            
             
            print_output(o,resul)
            
    else:
        print("{} y {} no son primos relativos. Error".format(det,mod))

In [657]:
k = [[11,8], [3,7]]
hill("-D",26, 2,"matriz_k.txt","matriz_cifrada.txt" ,0)

2
Cadena: holaquetal


In [658]:

hill("-C",26,2, "matriz_k.txt", 0,"resulta_hill.txt" )

2
hola que tal
Cadena: hola que tal
[array([15., 24.]), array([17., 10.]), array([2., 8.]), array([23.,  9.]), array([ 7., 25.])]


In [659]:
hill("-D",26, 2,"matriz_k.txt","resulta_hill.txt",0)

2
Cadena: holaquetal


## 2.b Método de Vigenere

El siguiente programa implementa el método. de Vigenere.

Llamada a la función: 

vigenere {-C|-D} {-k clave} [-i filein] [-o fileout]

-k cadena de caracteres usada como clave

In [165]:
def vigenere(modo,k,i=0,o=0):
    alfabeto='abcdefghijklmnopqrstuvwxyz'
    base=len(alfabeto)
    n=len(k)
    #Traducimos la clave de caracteres a números
    k_numerica=[]
    for j in k:
        if j in alfabeto: 
            k_numerica.append(alfabeto.index(j))
    if modo=="-C":
        cadena=read_input(i)
        # Traducimos los caracteres a números
        cadena_numerica=[]
        for k in cadena:
            if k in alfabeto: 
                cadena_numerica.append(alfabeto.index(k))
        # Dividimos en bloques de n elementos el input
        # Si m no es múltiplo de n se añade padding
        m=len(cadena_numerica)/n
        maxi=len(cadena_numerica)
        matriz_numerica=np.zeros((math.ceil(m),n))
        pos=0
        for i in range(math.ceil(m)):
            for j in range(n):
                if pos<maxi:
                    matriz_numerica[i][j]=cadena_numerica[pos]
                    pos=pos+1
        # Tenemos una matriz que tenemos que cifrar. 
        # Cada bloque es una fila de la matriz
        filas=matriz_numerica.shape[0]
        elementos=matriz_numerica.shape[1]
        #print("Matriz numerica")
        #print(matriz_numerica)
        matriz_cifrada=np.zeros((filas,elementos))
        #print("Matriz cifrada de ceros")
        #print(matriz_cifrada)
        if elementos==n:
            print("Bloques ok")
        for i in range(filas):
            for j in range(elementos):
                matriz_cifrada[i][j]=(matriz_numerica[i][j]+k_numerica[j])%base
        #print("Matriz cifrada")
        #print(matriz_cifrada)
        cadena_cifrada=np.concatenate(matriz_cifrada)
        print(cadena_cifrada)
        resul=""
        for i in cadena_cifrada:
            resul=resul+alfabeto[int(i)]
        #print(cadena_cifrada)
        print_output(o,resul)
    elif modo=="-D":
        cadena=read_input(i)
        #cadena_cifrada=cadena_cifrada.split(" ")
        #for i in range(len(cadena_cifrada)):
        #   cadena_cifrada[i]=int(cadena_cifrada[i])
        cadena_cifrada=[]
        for k in cadena:
            if k in alfabeto: 
                cadena_cifrada.append(alfabeto.index(k))
        print(cadena_cifrada)
        # Dividimos en bloques de n elementos el texto cifrado
        # Si m no es múltiplo de n se añade padding
        m=len(cadena_cifrada)/n
        maxi=len(cadena_cifrada)
        matriz_cifrada=np.zeros((math.ceil(m),n))
        pos=0
        for i in range(math.ceil(m)):
            for j in range(n):
                if pos<maxi:
                    matriz_cifrada[i][j]=cadena_cifrada[pos]
                    pos=pos+1
        # Tenemos una matriz que tenemos que descifrar. 
        # Cada bloque es una fila de la matriz
        filas=matriz_cifrada.shape[0]
        elementos=matriz_cifrada.shape[1]
        matriz_descifrada=np.zeros((filas,elementos))
        if elementos==n:
            print("Bloques ok")
        for i in range(filas):
            for j in range(elementos):
                matriz_descifrada[i][j]=(matriz_cifrada[i][j]-k_numerica[j])%base
        cadena_descifrada=np.concatenate(matriz_descifrada)
        cadena_texto=""
        for i in range(len(cadena_descifrada)):
            #cadena_cifrada[i]=int(cadena_cifrada[i])
            cadena_texto=cadena_texto+alfabeto[int(cadena_descifrada[i])]
        print_output(o,cadena_texto)

In [716]:
vigenere("-C", "clave")

Miren vuestras mercedes también cómo el emperador vuelve las espaldas y deja despechado a don Gaiferos, el cual ya ven cómo arroja, impaciente de la cólera, lejos de sí el tablero y las tablas, y pide apriesa las armas, y a don Roldán su primo pide prestada su espada Durindana, y cómo don Roldán no se la quiere prestar, ofreciéndole su compañía en la difícil empresa en que se pone; pero el valeroso enojado no lo quiere aceptar; antes, dice que él solo es bastante para sacar a su esposa, si bien estuviese metida en el más hondo centro de la tierra; y con esto, se entra a armar, para ponerse luego en camino. Vuelvan vuestras mercedes los ojos a aquella torre que allí parece, que se presupone que es una de las torres del alcázar de Zaragoza, que ahora llaman la Aljafería; y aquella dama que en aquel balcón parece, vestida a lo moro, es la sin par Melisendra, que desde allí muchas veces se ponía a mirar el camino de Francia, y puesta la imaginación en París y en su esposo, se consolaba en 

In [49]:
vigenere("-D", "clave")


rllvftlsve
Cadena: rllvftlsve
[17, 11, 11, 21, 5, 19, 11, 18, 21, 4]
Bloques ok
Cadena: palabrasaa


In [50]:
vigenere("-C", "clave", "texto_vigenere.txt", "resultado_vigenere.txt")

Cadena: Universidad Autonoma de Madrid
Bloques ok
[15. 19. 21. 25. 21. 20. 19.  3. 21.  7. 22.  4. 14.  8. 18. 14. 11.  3.
 25.  4.  5.  2.  8. 24.  4.]


In [51]:
vigenere("-D", "clave", "resultado_vigenere.txt", "descifrado_vigenere.txt")

Cadena: p t v z v u t d v h w e o i s o l d z e f c i y e
[15, 19, 21, 25, 21, 20, 19, 3, 21, 7, 22, 4, 14, 8, 18, 14, 11, 3, 25, 4, 5, 2, 8, 24, 4]
Bloques ok


## 2.c Criptoanálisis del cifrado de Vigenere

El primer paso para criptoanalizar el cifrado de vigneres es calcular el tamaño de la clave. Para ello hay dos opciones:

1-test de Kasiski: se trata de buscar repeticiones de conjuntos de caracteres en el texto y despues medir la distancia entre ellas.

2-indice de coincidencia: este índice determina la probabilidad de que 

Lo más interesante a destacar del uso de este índice es que si una cadena o texto descifrado posee un IC parecido al de su idioma, significa que la clave propuesta es la original del cifrado.


El siguiente programa implementa el test de kasiski.

Llamada a la función: 
kasiski {lista}{-tam conjunto_caracteres} [-i f ilein] [-o f ileout]

-tam longitud del conjunto.

In [795]:
def test_kasiski(lista, tam, i=0, o=0):
    from collections import Counter
    
    conj_dist=[]
    conjuntos=[]
    #buscamos conjuntos que se repitan en el texto y la distancia entre ellos
    i = 0
    while i < len(lista): 
        conj= lista[i:i+3] # Cogemos al menos 3 caracteres como tamaño de la tupla
        t = len(conj)
        if t == tam: #tiene que ser t, si no estamos al final de la lista
            for j in range(i+1,len(lista)): 
                if lista[i:i+t] == lista[j:j+t]: #Si coinciden, seguimos comprobando
                    while lista[i:i+t] == lista[j:j+t]:
                        t = t + 1
                    t = t -1
                    conj = lista[i:i+t] #Ahora tenemos un conjunto que sabemos que se repite
                    conjuntos.append(conj)
                    dist = j - i #calculamos la distancia
                    conj_dist.append([conj,dist])
                    
                    j = j + t + 1
            i = i + t -3 +1
        else:
            i = i + 1
            
    #escogemos el conjunto que más se repite y buscamos el mcd de sus distancias
    #
    res=0
    cont_conj=Counter(conjuntos) #contamos las repeticiones de los conjuntos
    mas_repetido=cont_conj.most_common(1) #nos quedamos con el que más se repite
    distancias=[]
    #elejimos los 5 más repetidos
    for i in range(len(conj_dist)):
        
        if conj_dist[i][0]==mas_repetido[0][0]:
                distancias.append(conj_dist[i][1])
        
    res=mcd_lista(distancias)
                
                
    return res
    

Función que cálcula el mcd de todos los elementos de una lista.

mcd_lista {lista}

In [796]:
def mcd_lista(lista):
    mcd=lista[0]
    for i in range(1,len(lista)):
        mcd=math.gcd(mcd,lista[i])
    return mcd

In [797]:
lista="kceizwpsovcdmzvepdzwvlmwmpnmjinpmkitldjvxfegzgwaniuaaghcdyyilldzwrpccefzayspliaitzszpefagccgeigozamvquadqrlcdipeeyinlcgitllznqddzwgwtvfnprjcnlsoedwancrtdzercizwcwanetxanccooisnonnyrcihsrtdzttpsoeflspiuaayewciihcyatgozdjrqwdirqdegesfizvgarzwvlrjjtpcdrfzlzwwnohtcleipcoiagkwehttpsvipbuzwgaoiirprjingagitzsjipzjvhqyogssfizvglcztvlrvrvpsymepqpindogsgdbvwvlnoirlrvwcnameufentqdanmdteiiueuqmgdehivtdvipplhwjznysepnovqoegevtemvcjcjrgdtjwgpnovclamqccpvvcaoiitdegygrozrelmdrqfegzcyvpiuervwoprxifpsgsuzjjwclqpinwaostcelygllgtccexisfenircenyrznzuwpenypldzpcdtjvtpsyinllxdccdzetlgjdcbuzejzrvpnlmvrnlleehprvccbuzpnldvqcbuziplqpinmaggpaamiepvzwvtdvenzmjvqpsgeutnketpldwgydmesfeyiuoevpnxuxlcdvzggdsztqyavqkcaminnahmpzdzvcycdeaauzwvllvmolgdrcniiiplrncgyspiuaonsupcjruzlvfcpnnyeluomxprdskceixcxbdrwynpixzcvwqbuzejzrvwwneyisfiurqginxquahw"
print(test_kasiski(lista,3))
#print(vigenere("-D", "clave"))

5


In [169]:
#función que cálcula los divisores de un número
def calcular_divisores(n):
    lista = []
    for i in range(2,n):
        if n % i == 0:
            lista.append(i)
    return lista

In [170]:
n=35
print(calcular_divisores(n))

[5, 7]



INDICE DE COINCIDENCIA

En general, el algoritmo consiste en iterar varias veces en busca de un tamaño n de clave correcto para un texto que nos dan cifrado. 

Este texto se va a dividir en bloques iguales al de clave propuesta, en este caso (n). En cada iteración, y luego, se van a
coger de cada bloque los caracteres cuyas posiciones puedan coincidir con la
subclave i de esta clave de tamaño n (como se ha visto en teoría). 

Cuabdo tenemos los n vectores, cálculamos sus IC. Si utilizando el conteo de frecuencias de sus caracteres, se aproximan al IC
del idioma utilizado, esto implica que el tamaño de clave n propuesto es el correcto para la
clave original del cifrado.

Esto ocurre porque se trata de un cifrado por
desplazamiento, por lo que las frecuencias de los caracteres permanece igual en el
estado de cifrado y de descifrado. 

El siguiente programa implementa el indice de coincidencia.

Llamada a la función: 
IC {lista}{-maxi maximo de iteraciones} [-i filein] [-o fileout]




In [806]:
def IC(cadena,maxi, i=0, o=0):
    from collections import Counter
    #INFORMACIÓN SOBRE EL IC DEL CASTELLANO Y EL ÍNGLES
    castellano=(['a', 11.96],['b', 0.92],['c', 2.92],['d', 6.87],['e', 16.78],['f', 0.52],['g', 0.73],
           ['h', 0.89],['i', 4.15],['j', 0.3],['k', 0.0],['l', 8.37],['m', 2.12],['n', 7.01],
           ['o', 8.69],['p', 2.77],['q', 1.53],['r', 4.94],['s', 7.88],['t', 3.31],['u', 4.80],
           ['v', 0.39],['w', 0.0],['x', 0.06],['y', 1.54],['z', 0.15])
    
    castellano_ord=sorted(castellano, key=lambda letra: letra[1], reverse=True)
    IC_castellano=0
    for i in range(len(castellano_ord)):
        IC_castellano= IC_castellano+castellano_ord[i][1]*castellano_ord[i][1]
    print(IC_castellano)
    IC_castellano=0.083
    
    ingles=(['a', 8.04],['b', 1.54],['c', 3.06],['d', 3.99],['e', 12.51],['f', 2.30],['g', 1.96],
        ['h', 5.49],['i', 7.26],['j', 0.16],['k', 0.67],['l', 4.14],['m', 2.53],['n', 7.09],
       ['o', 7.60],['p', 2.0],['q', 0.11],['r', 6.12],['s', 6,54],['t', 9.25],['u', 2.71],
       ['v', 0.99],['w', 1.92],['x', 0.19],['y', 1.73],['z', 0.19])

    ingles_ord=sorted(ingles, key=lambda letra: letra[1], reverse=True)
    IC_ingles=0
    for i in range(len(ingles)):
        IC_ingles= IC_ingles+(ingles[i][1]*ingles[i][1])
    
    IC_aleatorio=0.038
    #probamos con 18 tamaños diferentes (ponemos un máximo para que sea mas eficiente/posible)
    sublista=[]
    tamaño_clave=0
    #n es el tamaño de la clave propuesta
    for n in range(1,maxi):
        dic=dividir_lista(n,cadena) #dividimos el texto en sublistas del tamaño de la clave propuesta
        media=0
        for j in range(len(dic)): 
            media=media+calcular_IC(dic[j])
        print(media)
        #Si n es el tamaño de bloque con el que se ha cifrado, todos estos vectores tienen estructura de lenguaje 
        #y por tanto su coincidencia es diferente al de un lenguaje aleatorio y casi la misma que la del castellano
        #en este caso
        if media>=IC_castellano-0.005:
            if media<=IC_castellano+0.005: #buscamos un resultado aproximado
                    print("coincide, el tamaño de la clave es:")
                    print(media)
                    print(n)
                    sublista=dic
                    tamaño_clave=n
                    #break
                    #break
    return sublista, tamaño_clave
    

In [808]:
lista="jkceizwpsovcdmzvepdzwvlmwmpnmjinpmkitldjvxfegzgwaniuaaghcdyyilldzwrpccefzayspliaitzszpefagccgeigozamvquadqrlcdipeeyinlcgitllznqddzwgwtvfnprjcnlsoedwancrtdzercizwcwanetxanccooisnonnyrcihsrtdzttpsoeflspiuaayewciihcyatgozdjrqwdirqdegesfizvgarzwvlrjjtpcdrfzlzwwnohtcleipcoiagkwehttpsvipbuzwgaoiirprjingagitzsjipzjvhqyogssfizvglcztvlrvrvpsymepqpindogsgdbvwvlnoirlrvwcnameufentqdanmdteiiueuqmgdehivtdvipplhwjznysepnovqoegevtemvcjcjrgdtjwgpnovclamqccpvvcaoiitdegygrozrelmdrqfegzcyvpiuervwoprxifpsgsuzjjwclqpinwaostcelygllgtccexisfenircenyrznzuwpenypldzpcdtjvtpsyinllxdccdzetlgjdcbuzejzrvpnlmvrnlleehprvccbuzpnldvqcbuziplqpinmaggpaamiepvzwvtdvenzmjvqpsgeutnketpldwgydmesfeyiuoevpnxuxlcdvzggdsztqyavqkcaminnahmpzdzvcycdeaauzwvllvmolgdrcniiiplrncgyspiuaonsupcjruzlvfcpnnyeluomxprdskceixcxbdrwynpixzcvwqbuzejzrvwwneyisfiurqginxquahwqgezrcbuzpozrjuwpcvpnlnymezykeuttjerlsjtwpsosgwdzhqpngedzcvwgwlzkcaompcdentcwdvwfpegmupnyvcfenqkceigozlvhcfnwiuzeiqkeayhgwonpcmijwawakvkpsvuwpegpcdeyecpsxyrtrtentmkmtdegsunoipcmlvrelmvrildzwwnahmulyxqqdegeopnoeadevvtlnxefppzwccspwjprhsuzsxedplgsunohsutegpqdtpzkprvrnlcpprldzpollzjknijmtpnoeomiigozalygwgmexpmjvqbuziueeiesfegpqdcjvtpdjvgdenincetetdigmqoevrufevinnuvprzrcedprqmueogekysjpgycdefplhstzppiueolygprvypaammgytzcicaitttvvhqdutsnpmvrfwuzkqarzrfprtuwplzhgydjgkpnosulzjxgdlgixydjpgaompcdcvpnpsvgqdtpqdcayeuoegeetuyeflave"
IC(lista,18)

832.3528
0.04602705524285091
0.09453937592867756
0.14165229308267158
0.19540644593902043
0.4126098133865986
0.29782867107979216
0.34319360424635365
0.4048369870571621
0.4615539028961848
0.851535970938956
0.5796271023271132
0.630081092470473
0.6535203327036564
0.7488017724724181
1.2926852797639314
0.8735378488744897
0.9185491723466407


([], 0)

In [809]:
def calcular_IC(lista):
    #print(lista)
    alfabeto='abcdefghijklmnopqrstuvwxyz'
     #esta funcion esta mal
    castellano=(['a', 11.96],['b', 0.92],['c', 2.92],['d', 6.87],['e', 16.78],['f', 0.52],['g', 0.73],
           ['h', 0.89],['i', 4.15],['j', 0.3],['k', 0.0],['l', 8.37],['m', 2.12],['n', 7.01],
           ['o', 8.69],['p', 2.77],['q', 1.53],['r', 4.94],['s', 7.88],['t', 3.31],['u', 4.80],
           ['v', 0.39],['w', 0.0],['x', 0.06],['y', 1.54],['z', 0.15])
    
    from collections import Counter
    resul_contador = Counter(lista)
    
    frecuencias=[]
    for i in range(len(alfabeto)):
        frecuencias.append(resul_contador[alfabeto[i]])
   
    n_pares_iguales=0
    for i in range(len(frecuencias)):
        n_pares_iguales=n_pares_iguales+(frecuencias[i]*frecuencias[i]-1)
         #n_pares_iguales=castellano[i][1]*castellano[i][1]
    #n_pares_letras=(len(lista)*len(lista)-1)/2
    n_pares_letras=len(lista)*(len(lista)-1)
    #frecuencias=[]
    #el número de casos posibles en los que podemos elegir dos caracteres 
    #iguales entre un total de m caracteres del alfabeto
    #for j in range(len(resul_contador)): #vamos a calcular la frecuencia de cada caracter
            #frecuencias.append(resul_contador[j])
    
    #print(resul_contador)
    #print(frecuencias)
    #castellano_ord=sorted(castellano, key=lambda letra: letra[1], reverse=True)
    IC=n_pares_iguales/n_pares_letras
    #for i in range(len(lista)):
       # for j in range(len(castellano)):
           # if lista[i]==resul_contador[j][0]:
           #     IC= IC+(resul_contador[j][1]*resul_contador[j][1])
    return IC

In [724]:
def criptoanalisis_vigenere(lista, i=0):
    maxi=18 #la cantidad de intentos que queramos que haga como maximo
    conj=3 #la longitud de los conjuntos que queremos que compruebe
    sublista,n_ic= IC(lista,maxi)
    n_kasiski=test_kasiski(lista,conj)
    if n_ic==n_kasiski:
        print("el tamaño esta comprobado")
    else:
        print("puede que el tamaño escogido sea incorrecto")
        
        
    #ya sabemos la longitud de la clave, para que sea mas exacto, puesto que dependiendo
    #de la longitud del texto el IC calculado varia bastante, podemos calcular el tamaño
    #con el test de kasiski y comprobar si son el mismo resultado
    
    #ahora que ya tenemos el tamaño de la clave, vamos a esimar la clave 
    #de cifrado
    IC_castellano=0.078
    
    clave=[]
    print(len(clave))
    for i in range(len(sublista)):
        resul_contador = Counter(cadena)
        frecuencias=[]
        for s in range(len(alfabeto)):
            frecuencias.append(resul_contador[alfabeto[s]])
        
        for j in range(len(sublista[i])):
            for k in range(len(castellano)):
                
                if sublista[i][j]==castellano[k][0]:
                    m=castellano[k][1]*((frecuencias[k]-i)/len(sublista[i]))
                    print(m/10)
                    
                    if m/10>=IC_castellano-0.005:
                        if m/10<=IC_castellano+0.005:
                            print(sublista[i])
                            print(clave)
                            clave.append(sublista[i][j])
                            break
                            break
       # print(m)           
    print(clave)
        

In [794]:
lista="kceizwpsovcdmzvepdzwvlmwmpnmjinpmkitldjvxfegzgwaniuaaghcdyyilldzwrpccefzayspliaitzszpefagccgeigozamvquadqrlcdipeeyinlcgitllznqddzwgwtvfnprjcnlsoedwancrtdzercizwcwanetxanccooisnonnyrcihsrtdzttpsoeflspiuaayewciihcyatgozdjrqwdirqdegesfizvgarzwvlrjjtpcdrfzlzwwnohtcleipcoiagkwehttpsvipbuzwgaoiirprjingagitzsjipzjvhqyogssfizvglcztvlrvrvpsymepqpindogsgdbvwvlnoirlrvwcnameufentqdanmdteiiueuqmgdehivtdvipplhwjznysepnovqoegevtemvcjcjrgdtjwgpnovclamqccpvvcaoiitdegygrozrelmdrqfegzcyvpiuervwoprxifpsgsuzjjwclqpinwaostcelygllgtccexisfenircenyrznzuwpenypldzpcdtjvtpsyinllxdccdzetlgjdcbuzejzrvpnlmvrnlleehprvccbuzpnldvqcbuziplqpinmaggpaamiepvzwvtdvenzmjvqpsgeutnketpldwgydmesfeyiuoevpnxuxlcdvzggdsztqyavqkcaminnahmpzdzvcycdeaauzwvllvmolgdrcniiiplrncgyspiuaonsupcjruzlvfcpnnyeluomxprdskceixcxbdrwynpixzcvwqbuzejzrvwwneyisfiurqginxquahwqgezrcbuzpozrjuwpcvpnlnymezykeuttjerlsjtwpsosgwdzhqpngedzcvwgwlzkcaompcdentcwdvwfpegmupnyvcfenqkceigozlvhcfnwiuzeiqkeayhgwonpcmijwawakvkpsvuwpegpcdeyecpsxyrtrtentmkmtdegsunoipcmlvrelmvrildzwwnahmulyxqqdegeopnoeadevvtlnxefppzwccspwjprhsuzsxedplgsunohsutegpqdtpzkprvrnlcpprldzpollzjknijmtpnoeomiigozalygwgmexpmjvqbuziueeiesfegpqdcjvtpdjvgdenincetetdigmqoevrufevinnuvprzrcedprqmueogekysjpgycdefplhstzppiueolygprvypaammgytzcicaitttvvhqdutsnpmvrfwuzkqarzrfprtuwplzhgydjgkpnosulzjxgdlgixydjpgaompcdcvpnpsvgqdtpqdcayeuoegeetuyeflave"
criptoanalisis_vigenere(lista, 0)

832.3528
coincide, el tamaño de la clave es:
0.08333333333333334
2
puede que el tamaño escogido sea incorrecto
0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.01168
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.01168
0.0
0.0
0.0
0.0
-0.0006000000000000001
-0.0015600000000000002
-0.0
-0.013240000000000002
-0.011080000000000001
-0.0006000000000000001
-0.0012000000000000001
0.0
0.04784000000000001
-0.0061600000000000005
-0.06712
-0.0061600000000000005
-0.019200000000000002
-0.011080000000000001
-0.0015600000000000002
-0.013240000000000002
-0.0166
0.0
0.04784000000000001
-0.0061600000000000005
-0.0061600000000000005
-0.0012000000000000001
-0.0015600000000000002
-0.027479999999999997
-0.0
[]


In [61]:
def dividir_lista(n,lista): #ahora que sabemos la cardinalidad de la llave, dividimos la lista en ese modulo
    dic = {}
    for elem in range(n): #hacemos sublistas de n elementos
        dic[elem] = []
        
    i = 0
    for j in range (len(lista)):
        if i == n: #si el inidice es igual a n hemos llegado al final de la sublista
            i = 0 
        dic[i].append(lista[j])
        i = i + 1
    return dic

In [306]:
def descifrar_v(lista,dist):
    alfabeto="abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ"
    lista1=list()
    for elem in lista:
        valor = alfabeto.index(elem) - int(dist)
        if valor < 0:
            lista1.append(256 + (valor % -256))
        else:
            lista1.append(valor)
    return lista1

In [307]:
lista=input()

cont,freq=obtener_tuplas(lista)
l=contOcc(freq)
res = dividir_lista(l[i][0], lista) 
res=contOcc(freq)
occ = contOcc(res[i]) 
print(res[1])
print(occ[0][0])
print(lista[0])
print(l)
descifrar_v(res[0],occ[0][0])

NameError: name 'contOcc' is not defined

In [310]:
lista="jzlvuwptvpnwaHedlpvvcnoIxcCtzuwptJhqGawmgyyLygwaHyutcvwkruzwqyaIhqnoHscytzw"
lista1="jzlviuEawerpnNepoozroFcCeunoNeuDowvgEoysgycJwcDqPipztDippmPgjzszrvtdJgqxozwvpeEitnixmqEoOenxeIxgtnOitxiIedweLygyoxspDiBsfpsxmhCaMtgCoLygllHipzsxsoAiGesFevpizeNenrove"

cont, freq=obtener_tuplas(lista)
print(cont)
print(freq)
criptoanalisis_vigenere(lista1)

uwpt	i:4	j:29	diff:25		Divisors:[5]
waH	i:11	j:46	diff:35		Divisors:[5, 7]
2
[5, 5, 7]
ipz	i:49	j:139	diff:90		Divisors:[2, 3, 5, 6, 9, 10, 15, 18, 30, 45]
Lyg	i:108	j:133	diff:25		Divisors:[5]
5


NameError: name 'occ' is not defined

## 3. Cifrado de flujo

In [ ]:
def rec_fib(n):
    if n > 1:
        return rec_fib(n-1) + rec_fib(n-2)
    return n

In [ ]:
# Generador de secuencia aleatoria
def generador_aleatorio(m,cont):
    k=(rec_fib(m)%m)*m*cont
    return k

In [ ]:
# Ejemplo de secuencia cifrante de 5 elementos para clave 14
m=14
for i in range(5):
    k=generador_aleatorio(m,i)
    print(k)

In [ ]:
# m es la clave
# n es el tamaño de la secuencia de claves
def flujo(modo,m,n,i=0,o=0):
    alfabeto='abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ'
    if modo=="-C":
        cadena=read_input(i)
        # Traducimos los caracteres a números
        cadena_numerica=[]
        for k in cadena:
            if k in alfabeto: 
                cadena_numerica.append(alfabeto.index(k))
        # Ciframos carácter a carácter
        cadena_cifrada=[]
        count=0
        for i in cadena_numerica:
            if count<n:
                k=generador_aleatorio(m,count)
            else:
                k=generador_aleatorio(m,count-n)
            cadena_cifrada.append(int(bin(i^k)[2:]))
            count=count+1
        print_output(o,cadena_cifrada)
    elif modo=="-D":
        cadena_cifrada=read_input(i)
        cadena_cifrada=cadena_cifrada.split(" ")
        cadena_descifrada=[]
        cadena_texto=[]
        for i in range(len(cadena_cifrada)):
            cadena_cifrada[i]=int(cadena_cifrada[i],2)
        count=0
        for i in cadena_cifrada:
            if count<n:
                k=generador_aleatorio(m,count)
            else:
                k=generador_aleatorio(m,count-n)
            cadena_descifrada.append((i^k))
            count=count+1
        for i in range(len(cadena_descifrada)):
            cadena_texto.append(alfabeto[int(cadena_descifrada[i])])
        print_output(o,cadena_texto)

In [ ]:
flujo("-C", 4,2)

In [ ]:
flujo("-D", 4,2)

In [ ]:
flujo("-C", 10,20)

In [ ]:
flujo("-D", 10, 20)

## 4. Producto de criptosistemas de permutación

In [ ]:
# k1: vector de m elementos que constituye la clave para el cifrado de permutación por filas
# k2: vector de n elementos que constituye la clave para el cifrado de permutación por columnas
def permutacion(modo,k1,k2,i=0,o=0):
    alfabeto='abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ'
    if modo=="-C":
        cadena=read_input(i)
        # Traducimos los caracteres a números
        cadena_numerica=[]
        for k in cadena:
            if k in alfabeto: 
                cadena_numerica.append(alfabeto.index(k))

        # Matriz numerica es la matriz m x n 
        m=len(k1)
        n=len(k2)
        maxi=len(cadena_numerica)
        matriz_numerica=np.zeros((m,n))
         
        pos=0
        for i in range(m):
            for j in range(n):
                if pos<maxi:
                    matriz_numerica[i][j]=cadena_numerica[pos]
                    pos=pos+1
        print(matriz_numerica)
        
        # Cifrado de permutación por filas
        matriz_cifrada1=np.zeros((m,n))
        for i in range(m):
            for j in range(n):
                matriz_cifrada1[i][j]=matriz_numerica[k1[i]][j]
        print(matriz_cifrada1)
                
        # Cifrado de permutación por columnas
        matriz_cifrada2=np.zeros((m,n))
        for i in range(m):
            for j in range(n):
                matriz_cifrada2[i][j]=matriz_cifrada1[i][k2[j]]
                
        print(matriz_cifrada2)
        print_output(o, matriz_cifrada2)
    elif modo=="-D":
        cadena_cifrada=read_input(i)
        cadena_cifrada=cadena_cifrada.split(" ")
        
         # Matriz numerica es la matriz m x n 
        m=len(k1)
        n=len(k2)
        maxi=len(cadena_cifrada)
        matriz_cifrada=np.zeros((m,n))
         
        pos=0
        for i in range(m):
            for j in range(n):
                if pos<maxi:
                    matriz_cifrada[i][j]=cadena_cifrada[pos]
                    pos=pos+1
        print(matriz_cifrada)
        
        # Desciframos cifrado de permutación por columnas
        matriz_descifrada2=np.zeros((m,n))
        for i in range(m):
            for j in range(n):
                matriz_descifrada2[i][k2[j]]=matriz_cifrada[i][j]
        
        print(matriz_descifrada2)
        
        # Desciframos cifrado de permutación por filas
        matriz_descifrada1=np.zeros((m,n))
        for i in range(m):
            for j in range(n):
                matriz_descifrada1[k1[i]][j]=matriz_descifrada2[i][j]
        print(matriz_descifrada1)
        
        cadena_descifrada=np.concatenate(matriz_descifrada1)
        cadena_texto=[]
        for i in range(len(cadena_descifrada)):
            #cadena_cifrada[i]=int(cadena_cifrada[i])
            cadena_texto.append(alfabeto[int(cadena_descifrada[i])])

        print_output(o, cadena_texto)

In [ ]:
k1=[3,2,4,1,0]
k2=[1,3,2,0]
permutacion("-C", k1, k2)

In [ ]:
permutacion("-D", k1, k2)